## Project Title  
**ContractIQ — Instant Contract Summaries & Risk Scanner (BigQuery Generative AI)**  

### Problem Statement  
Enterprises and legal teams must review thousands of contracts and exhibits (NDAs, SOWs, vendor agreements). Manual review is slow, error prone, and expensive. ContractIQ ingests contracts (PDFs/text), extracts the clauses, and — using BigQuery’s native Generative AI — produces plain-English summaries, clause-level risk tags, and a contract risk score with a single SQL call.

### Impact Statement  
ContractIQ reduces contract triage time from hours/days to seconds, enabling faster deal reviews, automated red-flag alerts for high-risk clauses, and measurable reductions in legal review costs.


## High-Level Architecture

**Data Flow**

1. **Data Sources**  
   Public contract PDFs (EDGAR filings, CUAD dataset, etc.)

2. **Ingestion & Storage**  
   - Raw PDFs/Text → **Google Cloud Storage (GCS)**  
   - Loaded as **BigQuery Object Tables** or external tables  

3. **Pre-processing**  
   - OCR & document parsing via **Document AI / Cloud OCR**  
   - Output → **`contracts.raw`** table (contract_id, raw_text, metadata)  

4. **Generative AI Processing**  
   - BigQuery’s **`ML.GENERATE_TEXT` / `AI.GENERATE` / `AI.GENERATE_TABLE`** produce:  
     - `contracts.summaries` (one-paragraph summaries)  
     - `contracts.clauses` (clause_type + risk_level)  
     - `contracts.risk_scores` (numeric risk score)  

5. **Frontend / Demo**  
   - Streamlit app or Kaggle Notebook visualizes results.

### ASCII Architecture Diagram
```text
[PDFs (GCS)]
      |
      v
[BigQuery Object/External Tables]
      |
      v
[contracts.raw]
      |
      v
[AI.GENERATE / ML.GENERATE_TEXT]
      |
      v
[contracts.summaries, contracts.clauses, contracts.risk_scores]
      |
      v
[Streamlit demo / Notebook]


In [ ]:
# Cell 1: Import libraries and load partial dataset
import pandas as pd

# Path to the Kaggle SEC EDGAR dataset CSV
dataset_path = "/kaggle/input/sec-edgar-company-facts-september2023/companyfacts.csv"

# Load first 1000 rows only (memory safe)
df = pd.read_csv(dataset_path, nrows=1000)
print("Partial dataset loaded successfully!")
df.head()


In [ ]:
# Cell 2: Prepare contract index with demo PDF paths


# Create PDF path placeholder (for demonstration)
df['pdf_path'] = "/kaggle/working/edgar_pdfs/" + df['accn'].astype(str) + ".pdf"

# Select relevant columns for ContractIQ
contract_index = df[['accn', 'entityName', 'form', 'filed', 'pdf_path']].copy()
contract_index.rename(columns={
    'accn': 'contract_id',
    'entityName': 'company_name',
    'form': 'contract_type',
    'filed': 'date',
    'pdf_path': 'gcs_uri'
}, inplace=True)

# Preview the index
contract_index.head()


In [ ]:
# Cell 3: Save the partial contract index CSV
index_csv_path = "/kaggle/working/edgar_index_partial.csv"
contract_index.to_csv(index_csv_path, index=False)
print(f"Partial index CSV saved at: {index_csv_path}")


In [ ]:
# Cell 4: Create a folder to store downloaded PDFs
import os
pdf_folder = "/kaggle/working/edgar_pdfs"
os.makedirs(pdf_folder, exist_ok=True)
print(f"PDF folder ready at: {pdf_folder}")


In [ ]:
# Cell 5: Download first 20 PDFs (demo)
import requests

pdf_links = contract_index['gcs_uri'].tolist()[:20]  # first 20 for demo

for i, pdf_path in enumerate(pdf_links):
    # For demo, just create empty files (replace with real download if URLs available)
    open(pdf_path, 'wb').close()
    print(f"Prepared PDF placeholder: {pdf_path}")


In [ ]:
# Cell 6: Add a placeholder raw_text column (safe even if PDFs are empty)
import pandas as pd

pdf_text_df = contract_index[['contract_id']].copy()
pdf_text_df['raw_text'] = ""   # placeholder for extracted text

pdf_text_df.head()



In [ ]:
# Cell 7: Merge extracted text with contract index
contract_index_with_text = pd.merge(contract_index, pdf_text_df, how='left', left_on='contract_id', right_on='contract_id')

# Save final CSV ready for ContractIQ
final_csv_path = "/kaggle/working/edgar_index_full.csv"
contract_index_with_text.to_csv(final_csv_path, index=False)
print(f"Final index CSV with text saved at: {final_csv_path}")
contract_index_with_text.head()


In [4]:
import pandas as pd

# Mock contract data
mock_contracts = [
    {"contract_id": "C001", "raw_text": "Company A will deliver 100 units monthly. Late delivery incurs a penalty of $1,000 per week."},
    {"contract_id": "C002", "raw_text": "Company B must pay Company C $50,000 upon project completion. Early termination requires 2-week notice."},
    {"contract_id": "C003", "raw_text": "The agreement terminates automatically if either party breaches confidentiality or data protection rules."},
    {"contract_id": "C004", "raw_text": "Company D is liable for indemnity in case of intellectual property infringement."}
]

# Create DataFrame
df_mock = pd.DataFrame(mock_contracts)

# Save CSV (still needed for workflow)
df_mock.to_csv("/kaggle/working/mock_contracts.csv", index=False)

# Display the DataFrame in the notebook
df_mock  # This will render a table in Kaggle notebook


,contract_id,raw_text
0,C001,Company A will deliver 100 units monthly. Late...
1,C002,"Company B must pay Company C $50,000 upon proj..."
2,C003,The agreement terminates automatically if eith...
3,C004,Company D is liable for indemnity in case of i...


In [5]:
!pip install transformers

from transformers import pipeline
import pandas as pd

# Load a lightweight text generation model (CPU mode)
generator = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", device=-1)

# Load your mock contract dataset
df_mock = pd.read_csv("/kaggle/working/mock_contracts.csv")
df_mock.head()


2025-09-22 11:27:16.742394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758540437.017748      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758540437.091647      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cpu


,contract_id,raw_text
0,C001,Company A will deliver 100 units monthly. Late...
1,C002,"Company B must pay Company C $50,000 upon proj..."
2,C003,The agreement terminates automatically if eith...
3,C004,Company D is liable for indemnity in case of i...


In [7]:
summaries = []

for _, row in df_mock.iterrows():
    prompt = f"Summarize the key obligations and highlight any risk clauses for this contract:\n\n{row['raw_text']}\n\nSummary:"
    
    summary = generator(
        prompt,
        max_new_tokens=100,   # generate up to 100 new tokens
        do_sample=True,
        truncation=True       # truncate long input if needed
    )[0]['generated_text']
    
    # Remove the prompt from the output (keep only the generated summary)
    summary = summary.replace(prompt, "").strip()
    
    summaries.append({"contract_id": row['contract_id'], "summary": summary})

df_summaries = pd.DataFrame(summaries)
df_summaries


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


,contract_id,summary
0,C001,- Company A is contracted to deliver 100 units...
1,C002,"- Company B: Pay $50,000 upon project completi..."
2,C003,This contract outlines obligations related to ...
3,C004,This contract obligates Company D to compensat...


In [8]:
clauses = []

for _, row in df_mock.iterrows():
    prompt = (
        f"Break this contract into individual clauses. "
        f"For each clause provide: clause_type and risk_level (Low, Medium, High).\n\n"
        f"{row['raw_text']}\n\nClauses:"
    )
    
    result = generator(
        prompt,
        max_new_tokens=150,
        do_sample=True,
        truncation=True
    )[0]['generated_text']
    
    # strip prompt text if it repeats
    clean_result = result.replace(prompt, "").strip()
    clauses.append({"contract_id": row['contract_id'], "clauses": clean_result})

df_clauses = pd.DataFrame(clauses)
df_clauses


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


,contract_id,clauses
0,C001,1. Contractual Penalty - $100 per week (Low)\n...
1,C002,1. Non-Compete: Low\n2. Confidentiality: Mediu...
2,C003,1. Confidentiality\n2. Data Protection\n3. Bre...
3,C004,1. Risk Level: Medium\n2. Type of Clause: Inde...


In [9]:
import re

risk_scores = []

for _, row in df_mock.iterrows():
    prompt = (
        f"Give a single risk score from 1 (low) to 10 (high) for this contract:\n\n"
        f"{row['raw_text']}\n\nRisk score:"
    )
    
    result = generator(
        prompt,
        max_new_tokens=20,
        do_sample=True,
        truncation=True
    )[0]['generated_text']
    
    clean_result = result.replace(prompt, "").strip()
    # extract first digit 1–10, default to 5 if nothing found
    digits = re.findall(r'\d+', clean_result)
    score = int(digits[0]) if digits else 5
    risk_scores.append({"contract_id": row['contract_id'], "risk_score": score})

df_risk_scores = pd.DataFrame(risk_scores)
df_risk_scores



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


,contract_id,risk_score
0,C001,8
1,C002,6
2,C003,8
3,C004,5


In [11]:
df_mock.to_csv("/kaggle/working/mock_contracts.csv", index=False)
df_summaries.to_csv("/kaggle/working/mock_summaries.csv", index=False)
df_clauses.to_csv("/kaggle/working/mock_clauses.csv", index=False)
df_risk_scores.to_csv("/kaggle/working/mock_risk_scores.csv", index=False)

print("All CSVs saved to /kaggle/working/ for download or GitHub push.")


All CSVs saved to /kaggle/working/ for download or GitHub push.


In [12]:
# Step 14: Merge all outputs for combined demo
df_demo = df_mock.merge(df_summaries, on='contract_id') \
                 .merge(df_clauses, on='contract_id') \
                 .merge(df_risk_scores, on='contract_id')
df_demo.head()


,contract_id,raw_text,summary,clauses,risk_score
0,C001,Company A will deliver 100 units monthly. Late...,- Company A is contracted to deliver 100 units...,1. Contractual Penalty - $100 per week (Low)\n...,8
1,C002,"Company B must pay Company C $50,000 upon proj...","- Company B: Pay $50,000 upon project completi...",1. Non-Compete: Low\n2. Confidentiality: Mediu...,6
2,C003,The agreement terminates automatically if eith...,This contract outlines obligations related to ...,1. Confidentiality\n2. Data Protection\n3. Bre...,8
3,C004,Company D is liable for indemnity in case of i...,This contract obligates Company D to compensat...,1. Risk Level: Medium\n2. Type of Clause: Inde...,5


In [14]:
# Step 15: Visualize contract risk scores
import plotly.express as px

fig_risk = px.bar(
    df_demo,
    x='contract_id',
    y='risk_score',
    text='risk_score',
    color='risk_score',
    color_continuous_scale='Reds',
    title='Contract Risk Scores (1-10)'
)
fig_risk.update_layout(yaxis=dict(title='Risk Score'), xaxis=dict(title='Contract ID'))
fig_risk.show()


In [15]:
# Step 16: Interactive clause table
import plotly.graph_objects as go

fig_clauses = go.Figure(data=[go.Table(
    header=dict(values=['Contract ID', 'Clauses & Risk Levels'],
                fill_color='lightblue', align='left'),
    cells=dict(values=[df_demo.contract_id, df_demo.clauses],
               fill_color='lavender', align='left'))
])
fig_clauses.update_layout(title='Clauses & Risk Levels per Contract')
fig_clauses.show()


In [16]:
# Step 17: Clause-level risk distribution
clause_list = []

for _, row in df_demo.iterrows():
    for line in row['clauses'].split('\n'):
        if "(" in line and ")" in line:
            clause_text = line.split(" (")[0].split(". ",1)[1]
            risk_level = line.split("(")[-1].replace(")","")
            clause_list.append({"contract_id": row['contract_id'], "clause": clause_text, "risk_level": risk_level})

df_clause_flat = pd.DataFrame(clause_list)

fig_clause_risk = px.histogram(
    df_clause_flat,
    x='risk_level',
    color='risk_level',
    title='Clause-Level Risk Distribution',
    text_auto=True
)
fig_clause_risk.update_layout(yaxis=dict(title='Number of Clauses'), xaxis=dict(title='Risk Level'))
fig_clause_risk.show()


In [17]:
# Step 18: Interactive summaries table
fig_summaries = go.Figure(data=[go.Table(
    header=dict(values=['Contract ID', 'Summary'], fill_color='lightgreen', align='left'),
    cells=dict(values=[df_demo.contract_id, df_demo.summary], fill_color='beige', align='left'))
])
fig_summaries.update_layout(title='Contract Summaries')
fig_summaries.show()
